In [457]:
import wtfutil
import colorsys
import cubehelix
import numpy as np
import math


def gcolor(n):
    def to_rgb(hsl):
        return colorsys.hls_to_rgb(hsl[0]/360, hsl[2], hsl[1])
    
    bases =[
        [(240,.35,.35),(240,.65,.60),(240,.35,.75)],
        [(80,.35,.35),(80,.55,.45),(80,.35,.75)],
        [(40,.35,.35),(45,.75,.45),(40,.35,.75)],
        [(0,.35,.35),(0,.65,.55),(0,.35,.75)],
        [(300,.35,.35),(300,.65,.50),(300,.35,.75)],
    ]
    [under, excess] = [(0,.0,.35), (0,.0,.85)]
    
    # normalize
    n = round(n*1000)/1000
    
    n = n*10.0
    
    if n < 0:
        return to_rgb(under)
    
    if n >= 10.0:
        return to_rgb(excess)
    
    cat = int(n//2)
    sub = (n)%1
    if n/2%1 < 0.5:
        a = bases[cat][0]
        b = bases[cat][1]
    else:
        a = bases[cat][1]
        b = bases[cat][2]

    ca = 1-sub
    cb = sub
    
    final = (
        ca * a[0] + cb * b[0],
        ca * a[1] + cb * b[1],
        ca * a[2] + cb * b[2]
    )
    return to_rgb(final)



def fn_apply_colorize(idx, reverse=True, cmap='gcolor'):
    HIGHEST=800
    if np.isnan(idx):
        return ''
    if reverse:
        idx = np.round(np.log(idx)*100)
    
    
    ch = cubehelix.cmap(minLight=.15,maxLight=.95,minSat=0,maxSat=0)
    if cmap == 'ch1':
        ch = cubehelix.cmap(minLight=.15,maxLight=.95)
    elif cmap == 'ch2':
        ch = cubehelix.cmap(startHue=240,endHue=-300,minSat=1,maxSat=2.5,minLight=.3,maxLight=.8,gamma=.9)
    elif cmap == 'gcolor':
        ch = gcolor
    
    color = ch(idx/HIGHEST)
    fill = f'rgb({color[0]*256:.0f},{color[1]*256:.0f},{color[2]*256:.0f})'
            
    return '''
    1font-size: 15px;
    text-shadow: 0px 0px 2px rgba(0, 0, 0, 1), 0px 0px 0.5px rgba(0, 0, 0, 1);
    min-width: 60px;
    color: rgba(255, 255, 255, .99);
    1font-weight: 8000;
    background: ''' + fill


class CBar():
    el = []
    
    def __init__(self, series, stop=1.0, start=0.0, ticks=5, cmap='gcolor', wmod=0, h=90, m=900, bg=(0.95,0.95,0.95), mid=False):
        el = list()
        
        HIGHEST = 800
        
        el.append(f'<svg width="{m}" height="{h}" xmlns="http://www.w3.org/2000/svg">')
        
        # background
        color = bg
        fill = f'rgb({color[0]*256:.0f},{color[1]*256:.0f},{color[2]*256:.0f})'
        d = f'M {0} {0} L {0} {h} L {m} {h} L {m} 0 Z'
        el.append(f'<path d="{d}"fill="{fill}" stroke-dasharray="5,5" stroke-width="0.5" stroke="grey"/>')
        
        # mine the start
        amine = wtfutil.mine_series(series, start)[1]
        
        # mine until stop
        amine = wtfutil.mine_series(amine, stop-start)[0]
        
        ch = cubehelix.cmap(minLight=.15,maxLight=.95,minSat=0,maxSat=0)
        if cmap == 'ch1':
            ch = cubehelix.cmap(minLight=.15,maxLight=.95)
        elif cmap == 'ch2':
            ch = cubehelix.cmap(startHue=240,endHue=-300,minSat=1,maxSat=2.5,minLight=.3,maxLight=.8,gamma=.9)
        elif cmap == 'gcolor':
            ch = gcolor
        
        # calculate the ratio
        ratio = m/(stop-start)
        start = 0.0
        
        min_width = math.ceil(h/200)
        max_width = h
        
        for idx, val in zip(series.index, amine):
            
            # determine thickness
            
            if 0:
                pass
            elif wmod == 1:
                #min_width = 4
                width = idx / HIGHEST * 0.916666 + 0.08333
                width *= max_width #
                #width = (max_width-min_width)/HIGHEST * idx + min_width
            elif wmod == 2:
                width = (max_width-min_width)/np.exp(HIGHEST/100) * np.exp(idx/100) + min_width
            elif wmod == 3:
                min_width = 0
                width = math.sqrt(np.exp(idx/100))/math.sqrt(np.exp(HIGHEST/100)) * (max_width-min_width) + min_width
            elif wmod == 4:
                width = (1 + 6.5*idx/HIGHEST + 7.5 * (idx/HIGHEST)**2)/15
                width *= max_width
            elif wmod == 5:
                d = 2
                s = sum((1 + i*d for i in range(6)))
                p = np.polyfit([0, 160, 800], [1, 1+d, s], 2)
                w = np.polyval(p, idx)/s
                width = w * max_width
                
            else:
                width=1/15*h
                

            
            # TODO: calculate wmod4: dh/dV using pi
            
            if val == 0.0:
                continue
            
            color = ch(idx/HIGHEST)
            fill = f'rgb({color[0]*256:.0f},{color[1]*256:.0f},{color[2]*256:.0f})'
            x0 = math.floor(start)
            x1 = math.ceil(start + ratio*val)
            
            y0 = math.floor((h - width) / 2)
            y1 = math.ceil((h + width) / 2)
            
            y0 = h - width
            y1 = h
            
            d = f'M {x0} {y0} L {x0} {y1} L {x1} {y1} L {x1} {y0} Z'            
            el.append(f'<path d="{d}" fill="{fill}"/>')
            #el.append(f'<rect x="{rx}" y="{ry}" width="{rw}" height="{rh}" fill="{fill}" />')
            start += ratio*val
        
        for i in range(1,ticks):
            el.append(f'<line x1="{i*m/ticks}" y1="0" x2="{i*m/ticks}" y2="{h}" stroke-dasharray="5,5" stroke-width="1" stroke="rgba(128,128,128,0.25)"/>')

        hticks=5
        for i in range(0,5):
            idx = i * 160
            #terms = [1, 1.6, 2.2, 2.8, 3.4, 4.]
            #term2 = [1, 2.6, 4.2, 5.8, 7.4, 9.]
            
            if wmod == 3:
                min_width = 0
                width = math.sqrt(np.exp(idx/100))/math.sqrt(np.exp(HIGHEST/100)) * (max_width-min_width) + min_width
            elif wmod == 4:
                width = (1 + 6.5*idx/HIGHEST + 7.5 * (idx/HIGHEST)**2)/15
                width *= max_width
            elif wmod == 5:
                d = 2
                s = sum((1 + i*d for i in range(6)))
                p = np.polyfit([0, 160, 800], [1, 1+d, s], 2)
                w = np.polyval(p, idx)/s
                width = w * max_width
                
            else:
                width = i*160 / HIGHEST * 0.916666 + 0.08333
                width *= max_width #
            
            y0 = math.floor((h - width) / 2)
            y1 = math.ceil((h + width) / 2)
            y0 = h-width

            el.append(f'<line x1="0" y1="{y0}" x2="{m}" y2="{y0}" stroke-dasharray="5,5" stroke-width="1" stroke="rgba(128,128,128,0.25)"/>')
#            el.append(f'<line x1="0" y1="{y1}" x2="{m}" y2="{y1}" stroke-dasharray="5,5" stroke-width="1" stroke="rgba(128,128,128,0.25)"/>')

        el.append(f'</svg>')
    
        self.el = el
        
    
    def _repr_html_(self):
        return ''.join(self.el)


In [458]:
#CBar(fa.mvs, wmod=3, h=500, m=500)

In [459]:
idx = list(range(800,-40,-40))
#idx = list(range(800,-1,-1))
ones = pd.Series(1/len(idx), index=idx)
CBar(ones *.90, 1, 0, 5, wmod=5, h=500, m=500)

In [460]:
idx = list(range(800,-40,-40))
#idx = list(range(800,-1,-1))
ones = pd.Series(1/len(idx), index=idx)
CBar(ones *.90, 1, 0, 5, wmod=4, h=500, m=500)

In [304]:
idx = list(range(800,-40,-40))
ones = pd.Series(1/len(idx), index=idx)
CBar(ones *.8, 1, 0, 5, cmap='ch2', wmod=4, h=500, m=500)

In [305]:
sample_mvs = pd.Series([0.00011275,
 5.65e-05,
 0.0,
 0.0009420000000000001,
 0.00343025,
 0.0050644999999999996,
 0.0036259999999999994,
 0.01298875,
 0.03919825,
 0.006884749999999999,
 0.020551250000000007,
 0.032798999999999995,
 0.030356000000000008,
 0.040853749999999994,
 0.12822425,
 0.18221775,
 0.025618499999999992,
 0.07528474999999998,
 0.012026750000000001,
 0.012269000000000002,
 0.01165975], index=list(range(800,-40,-40)))

In [476]:
CBar(sample_mvs, 1, 0, 5, wmod=2, h=400, m=400)

In [403]:
d=1.0
s = sum((1 + i*d for i in range(6)))
s

21.0

In [374]:
(1+1+d)/s

0.3333333333333333

In [404]:
np.polyfit([0, 160, 800], [1, 1+1, 21], 2)

array([  2.92968750e-05,   1.56250000e-03,   1.00000000e+00])

In [383]:
np.polyfit([0, 160, 800], [1/15, (1+1.6)/15, 15/15], 2)

array([  7.81250000e-07,   5.41666667e-04,   6.66666667e-02])

In [399]:
np.polyval(np.polyfit([0, 160, 800], [1, 2, 6], 1), 160)

2.0000000000000009

In [410]:
np.polyval(np.polyfit([0, 160, 800], [1, 1+1, 21], 2), 800-160)/21

0.6666666666666663